In [1]:
import pandas as pd
import ast
import operator

In [2]:
df = pd.read_csv("../resources/count_vectors_all.csv")

In [77]:
# Refernce: https://www.tutorialspoint.com/How-to-convert-a-string-to-dictionary-in-Python, 15.1.2021.
# Reference: https://stackoverflow.com/questions/20577840/python-dictionary-sorting-in-descending-order-based-on-values, 15.1.2021.
# Reference: https://www.stechies.com/valueerror-using-scalar-values-must-pass-index/, 15.1.2021.

topN = 10 # select the number of top entities
df_list = []
for i in range(0,df.shape[0]):
    df_tmp = ast.literal_eval(df.iloc[i, 1])
    df_tmp = sorted(df_tmp.items(), key=operator.itemgetter(1), reverse=True)
    df_tmp = dict(df_tmp)
    df_tmp = pd.DataFrame(df_tmp, index=[0])
    df_tmp = df_tmp.T
    df_tmp.columns = ['count']
    df_tmp.reset_index(inplace=True)
    df_tmp = df_tmp.rename(columns = {'index':'entity'})
    df_tmp["month"] = df.iloc[i, 0]
    topN_tmp = df_tmp.iloc[:(topN + 1),]
    df_list.append(topN_tmp)
df_plot = pd.concat(df_list, sort=False)

In [79]:
# define a list of clusters (result from clustering step)
BrexitCluster = ['2018-07', '2018-11', '2019-03', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12']
TonyBlairCluster = ['2001-02', '2001-03', '2001-05', '2004-10', '2005-04', '2005-05', '2005-10', '2006-09', '2006-10', '2007-05', '2007-06', '2007-07']
ObamaCluster = ['2008-02', '2008-10', '2008-11', '2008-12', '2009-01', '2009-02']
FootballCluster = ['2011-02', '2011-03', '2011-04', '2011-05']

In [80]:
in_cluster_list = FootballCluster

In [81]:
import plotly.express as px
import plotly.graph_objects as go
#Reference: https://stackoverflow.com/questions/63386812/plotly-how-to-hide-axis-titles-in-a-plotly-express-figure-with-facets, 15.1.2021.

fig = px.bar(df_plot[df_plot["month"].isin(in_cluster_list)], x="entity", y="count", color="entity",
             facet_row="month")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# hide subplot y-axis titles and x-axis titles
for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
        
# keep all other annotations and add single y-axis and x-axis title:
fig.update_layout(
    # keep the original annotations and add a list of new annotations:
    annotations = list(fig.layout.annotations) + 
    [go.layout.Annotation(
            x=-0.07,
            y=0.5,
            font=dict(
                size=14, color = 'black'
            ),
            showarrow=False,
            text="Counts",
            textangle=-90,
            xref="paper",
            yref="paper"
        )
    ] +
    [go.layout.Annotation(
            x=0.5,
            y=-0.2,
            font=dict(
                size=14, color = 'black'
            ),
            showarrow=False,
            text="Entities",
            textangle=-0,
            xref="paper",
            yref="paper"
        )
    ]
)

fig.update_layout(
    showlegend=False,
    font=dict(size=8)
)

fig.show()